<a href="http://landlab.github.io"><img style="float: left; height: 175px; width: 175px" src="../landlab_logo_picture.jpg"></a> <h3 style="margin: 117px 0 0 185px; font-weight: 300;">a toolkit for modeling earth surface processes</h3>

# Introduction to the Lithology and LithoLayers objects

Lithology and LithoLayers are two landlab components meant to make it easier to work with spatially variable lithology that produces spatially variable parameter values (e.g. stream power erodability or diffusivity). 

In this tutorial we will first use the LithoLayers to erode either dipping layeres or an anticline. Then we will use Lithology to create inverted topography. 

We will also use [xarray](https://xarray.pydata.org/en/stable/) to store and annotate our model output. 

To start, we will import the necessary modules. A note: this tutorial uses the [HoloViews package](http://holoviews.org) for visualization. This package is a great tool for dealing with multidimentional annotated data (e.g. an xarray dataset). If you get an error on import, consider updating dask (this is what the author needed to do in April 2018). You will also need to have the [Bokeh](https://bokeh.pydata.org/en/latest/) and [Matplotlib](https://matplotlib.org) packages installed.

In [ ]:
import os
import numpy as np
import xarray as xr
import dask

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

import holoviews as hv
hv.notebook_extension('matplotlib')

from landlab import RasterModelGrid
from landlab.components import FlowAccumulator, FastscapeEroder, LinearDiffuser, Lithology, LithoLayers
from landlab.plot import imshow_grid

## Part 1: Creating layered rock

First we will create an instance of a LithoLayers. Both LithoLayers and Lithology work closely with a landlab ModelGrid and stores information about rock type at each node. 

To create LithoLayers you need the following information:

1. A model grid that has the field `'topographic__elevation'` already created.
2. A dictionary of rock property attributes that maps. 
3. A list of elevations, alled `z0` that your layers will go through at specified anchor point (default value for the anchor point is (0, 0). When `z0` is negative.... TODO.
4. A functional form in two variables (x and y) that defines the shape of your surface. 

The use of this function form makes it possible for any function of x and y to be passed to LithoLayers.
In this example the option for an anticline is currently uncommented, though options for steep and shallow dipping layers are also provided. 

In this tutorial we will first make an example to help build intuition. If you are interested in reading more... [LithoLayers](http://landlab.readthedocs.io/en/release/landlab.layers.litholayers.html)

First, we create a RasterModelGrid with topography initialized with a small amount of random noise. 

Make a very small grid, with a very easy example of layers varying, and illustrate that first so people get their confidence up. I know people can read the docstrings, but I think the point of the notebook is that people should not need to read the docstring with the notebook?

In [ ]:
mg = RasterModelGrid((100, 60), 200)
z = mg.add_zeros('node', 'topographic__elevation') 
random_field = 0.01*np.random.randn(mg.size('node'))
z += random_field - random_field.min()

Next we.... 

Desribe attrs, z0s, bottom layer, and IDS. say what tile is. 

Remind people what it means when z0s are negative. 

In [ ]:
attrs = {'K_sp': {0: 0.0003,
                  1: 0.0001}}


z0s = 50 * np.arange(-20, 20)

# we create a bottom layer that is very thick. 
z0s[-1] = z0s[-2] + 10000


ids = np.tile([0,1], 20) 





TODO, describe making functions

In [ ]:
func = lambda x, y : ((0.001*x)+(0.003*y))



Finally we construct our LithoLayers component. Make a plot and talk through it.  

## Part 2: Creation of a landscape evolution model with LithoLayers


Introductory text.


In [ ]:
mg = RasterModelGrid((100, 60), 200)
z = mg.add_zeros('node', 'topographic__elevation') 
random_field = 0.01*np.random.randn(mg.size('node'))
z += random_field - random_field.min()

attrs = {'K_sp': {0: 0.0003,
                  1: 0.0001}}


z0s = 50 * np.arange(-20, 20)

# we create a bottom layer that is very thick. 
z0s[-1] = z0s[-2] + 10000


ids = np.tile([0,1], 20) 



# Anticline
anticline_func = lambda x, y : ((0.002*x)**2+(0.001*y)**2)

# Shallow dips
shallow_func = lambda x, y : ((0.001*x)+(0.003*y))

# Steeper dips
steep_func = lambda x, y : ((0.01*x)+(0.01*y))

Intervening text.

In [ ]:
# Anticline
lith = LithoLayers(mg, z0s, ids, x0=6000, y0=10000, function=anticline_func, attrs=attrs)

# Shallow dips
#lith = LithoLayers(mg, z0s, ids, function=shallow_func, attrs=attrs)

# Steeper dips
#lith = LithoLayers(mg, z0s, ids, function=steep_func, attrs=attrs)

Now that we've created LithoLayers, model grid fields for each of the LithoLayers attributes exist and have been set to the values of the rock exposed at the surface. 

Here we plot the value of `'K_sp'` as a function of the model grid. 

In [ ]:
imshow_grid(mg, 'K_sp')

As you can see (in the default anticline option) we have concentric elipses of stronger and weaker rock. 

Next, lets instantiate a FlowAccumulator and a FastscapeEroder to create a simple landscape evolution model. 

We will point the FastscapeEroder to the model grid field `'K_sp'` so that it will respond to the spatially variable erodabilities created by LithoLayers. 

In [ ]:
nts = 500
U = 0.001
dt = 1000

fa = FlowAccumulator(mg)
sp = FastscapeEroder(mg, K_sp='K_sp')

Before we run the model we will also instatiate an xarray dataset used to store the output of our model through time for visualization. 

The next block may look intimidating, but I'll try and walk you through what it does. 

[xarray](https://xarray.pydata.org/en/stable/) allows us to create a container for our data and label it with information like units, dimensions, short and long names, etc.  xarray gives all the tools for dealing with N-dimentional data provided by python packages such as [numpy](http://www.numpy.org), the labeling and named indexing power of the [pandas](https://pandas.pydata.org) package, and the data-model of the [NetCDF file](https://www.unidata.ucar.edu/software/netcdf/).

This means that we can use xarray to make a "self-referetial" dataset that contains all of the variables and attributes that describe what each part is and how it was made. In this application, we won't make a fully self-referential dataset, but if you are interested in this, check out the [NetCDF best practices](https://www.unidata.ucar.edu/software/netcdf/docs/BestPractices.html). 

Important for our application is that later on we will use the [HoloViews package](http://holoviews.org) for visualization. This package is a great tool for dealing with multidimentional annotated data and will do things like automatically create nice axis labels with units. However, in order for it to work, we must first annotate our data to include this information.

Here we create an xarray Dataset with two variables `'topographic__elevation'` and `'rock_type__id'` and three dimensions `'x'`, `'y'`, and `'time'`. 

We pass xarray two dictionaries, one with information about the data variabiables (`data_vars`) and one with information about the coordinate system (`coords`). For each data variable or coordinate, we pass a tuple of three items: `(dims, data, atts)`. The first element is a tuple of the name of the dimensions, the second element is the data, an the third is a dictionary of attributes. 

In [ ]:
ds = xr.Dataset(data_vars={'topographic__elevation' : (('time', 'y', 'x'),  # tuple of dimensions
                                                       np.empty((nts, mg.shape[0], mg.shape[1])), # n-d array of data
                                                      {'units' : 'meters', # dictionary with data attributes
                                                       'long_name': 'Topographic Elevation'}),
                           'rock_type__id': (('time', 'y', 'x'), 
                                             np.empty((nts, mg.shape[0], mg.shape[1])),
                                            {'units' : '-',
                                             'long_name' : 'Rock Type ID Code'})},
                coords={'x': (('x'), # tuple of dimensions
                              mg.x_of_node.reshape(mg.shape)[0,:], # 1-d array of coordinate data
                              {'units' : 'meters'}), # dictionary with data attributes
                        'y': (('y'), 
                              mg.y_of_node.reshape(mg.shape)[:, 1],
                              {'units' : 'meters'}),
                        'time': (('time'),  
                                 dt*np.arange(nts)/1e6,
                                 {'units': 'millions of years since model start',
                                  'standard_name' : 'time'})})

We can print the data set to get some basic information about it.

In [ ]:
print(ds)

We can also print a single variable to get more detailed information about it. 

Since we initialized the datset with empty arrays for the two data variables, we just see zeros for the data values. 

In [ ]:
ds.topographic__elevation

Next, we run the model. In each time step we first run the FlowAccumulator to direct flow and accumulatate drainage area. Then the FastscapeEroder erodes the topography based on the stream power equation using the erodability value in the field`'K_sp'`. We create an uplift field that uplifts only the model grid's core nodes. After uplifting these core nodes, we update LithoLayers. Importantly, we must tell the LithoLayers how it has been advected upward by uplift. 

`lith.run_one_step` has an optional argument `rock_id` to use when some material may be deposited. Since we are using the FastscapeEroder which is fully detachment limited, we don't need to set this. 

Within each timestep we save information about the model for plotting. 

In [ ]:
out_fields = ['topographic__elevation',
              'rock_type__id']

for i in range(nts):
    fa.run_one_step()
    sp.run_one_step(dt = dt)
    dz_ad = np.zeros(mg.size('node'))
    dz_ad[mg.core_nodes] = U * dt
    z += dz_ad
    lith.run_one_step(dz_advection = dz_ad)
    
    for of in out_fields:
        ds[of][i,:,:] = mg['node'][of].reshape(mg.shape)

Now that the model has run, lets start by plotting the resulting topography. 

In [ ]:
imshow_grid(mg, 'topographic__elevation', cmap='viridis')

The layers of rock clearly influence the form of topography. 

Next we will use HoloViews to visualize the topography and rock type together. 

To start, we create a HoloViewDataset from our xarray datastructure. 

In [ ]:
hvds_topo = hv.Dataset(ds.topographic__elevation)
hvds_rock = hv.Dataset(ds.rock_type__id)
hvds_topo

Next we specify that we want two images, one showing rock type and one showing topographic elevation. A slider bar shows us model time in millions of years.  

Be patient. Running this next block may take a moment. HoloViews is rendering an image of all time slices so you can see an animated slider. This is pretty magical (but not instantaneous).  

In [ ]:
%opts Image style(interpolation='bilinear', cmap='viridis') plot[colorbar=True]
topo = hvds_topo.to(hv.Image, ['x', 'y'])
rock = hvds_rock.to(hv.Image, ['x', 'y'])

topo + rock

We can see the form of the anticline advecting through the topography. Cool!


## Part 3: Creation of Inverted Topography

Here we will explore making inverted topography by eroding Lithology with constant properties for half of the model evaluation time, and then filling Lithology in with resistant material only where the drainage area is large. This is meant as a simple example of filling in valleys with volcanic material. 

All of the details of the options for creating a [Lithology](http://landlab.readthedocs.io/en/release/landlab.layers.lithology.html) can be found here. 

In the next code block we make a new model and run it. Note that because we are using the LinearDiffuser which may result in depositing material we must specify a rock_id in the Lithology `run_one_step` method.

We also are handling the model grid boundary conditions differently than in the last example, setting the boundaries on the top and bottom to closed. 

In [ ]:
mg2 = RasterModelGrid((100, 60), 200)
mg2.set_closed_boundaries_at_grid_edges(False, True, False, True)
z2 = mg2.add_zeros('node', 'topographic__elevation') 
random_field = 0.01*np.random.randn(mg2.size('node'))
z2 += random_field - random_field.min()

thicknesses2 = [10000]
ids2 =[0]

attrs2 = {'K_sp': {0: 0.0001,
                   1: 0.00001},
          'D': {0: 0.4,
                1: 0.001}}

lith2 = Lithology(mg2, thicknesses2, ids2, attrs=attrs2)

nts = 500
U = 0.005
dt = 1000

fa2 = FlowAccumulator(mg2)
sp2 = FastscapeEroder(mg2, K_sp='K_sp')
ld2 = LinearDiffuser(mg2, linear_diffusivity='D')

out_fields = ['topographic__elevation',
              'rock_type__id']

out_fields = ['topographic__elevation',
              'rock_type__id']

nts = 500
U = 0.001
dt = 1000


ds2 = xr.Dataset(data_vars={'topographic__elevation' : (('time', 'y', 'x'), 
                                                        np.empty((nts, mg2.shape[0], mg2.shape[1])),
                                                       {'units' : 'meters',
                                                        'long_name': 'Topographic Elevation'}),
                            'rock_type__id': (('time', 'y', 'x'), 
                                              np.empty((nts, mg2.shape[0], mg2.shape[1])),
                                             {'units' : '-',
                                              'long_name' : 'Rock Type ID Code'})
                           },
                 coords={'x': (('x'), 
                               mg2.x_of_node.reshape(mg2.shape)[0,:],
                               {'units' : 'meters'}),
                         'y': (('y'), 
                               mg2.y_of_node.reshape(mg2.shape)[:, 1],
                               {'units' : 'meters'}),
                         'time': (('time'),  
                                  dt*np.arange(nts)/1e6,
                                  {'units': 'millions of years since model start',
                                   'standard_name' : 'time'})
                        }
                 )


half_nts = int(nts/2)
for i in range(half_nts):
    fa2.run_one_step()
    sp2.run_one_step(dt=dt)
    ld2.run_one_step(dt=dt)
    dz_ad2 = np.zeros(mg2.size('node'))
    dz_ad2[mg2.core_nodes] = U * dt
    z2 += dz_ad2
    lith2.run_one_step(dz_advection = dz_ad2, rock_id=0)
    
    for of in out_fields:
        ds2[of][i,:,:] = mg2['node'][of].reshape(mg2.shape)

After the first half of run time, let's look at the topography. 

In [ ]:
imshow_grid(mg2, 'topographic__elevation', cmap='viridis')

We can see that we have developed ridges and valleys as we'd expect from a model with stream power erosion and linear diffusion. 

Next we will create some volcanic deposits that fill the channels in our model.

In [ ]:
volcanic_deposits = np.zeros(mg2.size('node'))
da_big_enough = mg2['node']['drainage_area']>5e4

topo_difference_from_top = mg2['node']['topographic__elevation'].max() - mg2['node']['topographic__elevation']

volcanic_deposits[da_big_enough] = 0.25 * topo_difference_from_top[da_big_enough]
volcanic_deposits[mg2.boundary_nodes] = 0.0

z2 += volcanic_deposits
lith2.run_one_step(rock_id=1)

imshow_grid(mg2, volcanic_deposits)

We should expect that the locations of our valleys and ridges change as the river system encouters the much stronger volcanic rock. 

In [ ]:
for i in range(half_nts, nts):
    fa2.run_one_step()
    sp2.run_one_step(dt=dt)
    ld2.run_one_step(dt=dt)
    dz_ad2 = np.zeros(mg2.size('node'))
    dz_ad2[mg2.core_nodes] = U * dt
    z2 += dz_ad2
    lith2.run_one_step(dz_advection = dz_ad2, rock_id=0)
    
    for of in out_fields:
        ds2[of][i,:,:] = mg2['node'][of].reshape(mg2.shape)

Now that the model has run, let's plot the final elevation

In [ ]:
imshow_grid(mg2, 'topographic__elevation', cmap='viridis')

And now a HoloView Plot that lets us explore the time evolution of the topography

In [ ]:
hvds_topo2 = hv.Dataset(ds2.topographic__elevation)
hvds_rock2 = hv.Dataset(ds2.rock_type__id)

%opts Image style(interpolation='bilinear', cmap='viridis') plot[colorbar=True]
topo2 = hvds_topo2.to(hv.Image, ['x', 'y'])
rock2 = hvds_rock2.to(hv.Image, ['x', 'y'])

topo2 + rock2

In [ ]:
# if you wanted to output to visualize in something like ParaView, the following commands can be used
#ds.to_netcdf('anticline.nc')
#ds2.to_netcdf('inversion.nc')

Sure enough, the volcanic deposits impact the location of the ridges and valleys. The old valleys become ridges because it takes so much time for them to be eroded. 

You can explore how this changes as the thickness of the deposit changes and as the relative erodabilities change. 


## The end.

Nice work getting to the end of the tutorial!

For more detailed information about the [Lithology](http://landlab.readthedocs.io/en/release/landlab.layers.lithology.html) and [LithoLayers](http://landlab.readthedocs.io/en/release/landlab.layers.litholayers.html) objects, check out their detailed documentation. 

# **Click [here](https://github.com/landlab/landlab/wiki/Tutorials) for more Landlab tutorials**